In [1]:
import torch, sys
print("Python:", sys.version.split()[0])
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.10.18
Torch: 2.5.1+cu121
CUDA available: True
GPU: Tesla P100-PCIE-12GB


In [ ]:
pip install tensorflow>=2.12.0 pandas numpy scikit-learn opencv-python pillow matplotlib seaborn tqdm h5py scipy

In [2]:
# --- Closed-set 50k split for identity classification (CelebA)
from pathlib import Path
import pandas as pd
import numpy as np

rng = np.random.default_rng(42)

ROOT    = Path("/home/anand.ha/NNDL/celeba")
RAW     = ROOT / "data/raw"
IMG_DIR = RAW  / "img_align_celeba"
ID_PATH = RAW  / "identity_CElebA.csv"  # accept both spellings
if not ID_PATH.exists(): ID_PATH = RAW / "identity_CelebA.csv"

assert IMG_DIR.is_dir(), f"Images folder missing: {IMG_DIR}"
assert ID_PATH.exists(), f"Identity file missing: {ID_PATH}"

def load_identity(path: Path) -> pd.DataFrame:
    # 1) try CSV with header
    try:
        df = pd.read_csv(path)
        cols = [c.strip().lower() for c in df.columns]
        if {"image_id","identity"}.issubset(cols):
            df.columns = cols
            return df[["image_id","identity"]]
    except Exception:
        pass
    # 2) CSV w/o header
    try:
        df2 = pd.read_csv(path, header=None).iloc[:, :2]
        df2.columns = ["image_id","identity"]
        return df2
    except Exception:
        pass
    # 3) whitespace-separated
    df3 = pd.read_csv(path, sep=r"\s+", header=None, engine="python").iloc[:, :2]
    df3.columns = ["image_id","identity"]
    return df3

id_df = load_identity(ID_PATH)
id_df["image_id"] = id_df["image_id"].astype(str).str.strip()
id_df["identity"] = pd.to_numeric(id_df["identity"], errors="coerce")
id_df = id_df.dropna(subset=["identity"]).reset_index(drop=True)
id_df["identity"] = id_df["identity"].astype(int)

# keep only rows whose image actually exists
exists = id_df["image_id"].apply(lambda n: (IMG_DIR / n).is_file())
id_df = id_df[exists].reset_index(drop=True)

# identities with at least 3 images (so we can do train/val/test)
counts = id_df["identity"].value_counts()
eligible_ids = counts[counts >= 3].index.tolist()

TARGET = 50_000  # <<<<<<<< 50k total images

# choose identities (most frequent first) until total images >= TARGET
chosen_ids, total = [], 0
for ident, c in counts.loc[eligible_ids].items():
    chosen_ids.append(ident); total += c
    if total >= TARGET: break

subset = id_df[id_df["identity"].isin(chosen_ids)].copy()

# per-identity closed-set split ~80/10/10 with minimums
def split_one(g):
    n = len(g)
    idx = rng.permutation(n)
    n_val  = max(1, int(round(0.10*n)))
    n_test = max(1, int(round(0.10*n)))
    n_train = max(1, n - n_val - n_test)
    if n == 3:   n_train, n_val, n_test = 1,1,1
    elif n == 4: n_train, n_val, n_test = 2,1,1
    elif n == 5: n_train, n_val, n_test = 3,1,1
    tr = idx[:n_train]; va = idx[n_train:n_train+n_val]; te = idx[n_train+n_val:]
    out = g.copy()
    out.loc[out.index.isin(g.index[tr]), "split3"] = "train"
    out.loc[out.index.isin(g.index[va]), "split3"] = "val"
    out.loc[out.index.isin(g.index[te]), "split3"] = "test"
    return out

subset = subset.groupby("identity", group_keys=False).apply(split_one)

# trim to exactly TARGET, dropping from train first (but keep ≥1 train per identity)
def trim_to_target(df, target):
    excess = len(df) - target
    if excess <= 0: return df
    train = df[df["split3"]=="train"].copy()
    keep_min = train.groupby("identity").head(1).index
    droppable = train.index.difference(keep_min)
    drop_n = min(excess, len(droppable))
    drop_idx = rng.choice(droppable, size=drop_n, replace=False) if drop_n>0 else []
    return df.drop(index=drop_idx)

subset = trim_to_target(subset, TARGET)

# final labels 0..C-1
id_set   = sorted(subset["identity"].unique().tolist())
id2label = {iid:i for i, iid in enumerate(id_set)}
subset["label"] = subset["identity"].map(id2label).astype(int)

train_df = subset[subset["split3"]=="train"].reset_index(drop=True)
val_df   = subset[subset["split3"]=="val"].reset_index(drop=True)
test_df  = subset[subset["split3"]=="test"].reset_index(drop=True)
NUM_CLASSES = len(id_set)

def summarize(name, d):
    if len(d)==0: return f"{name}: n=0, ids=0"
    vc = d["identity"].value_counts()
    return f"{name}: n={len(d):5d}, ids={d['identity'].nunique():4d}, min/max per id={int(vc.min())}/{int(vc.max())}"

print("Closed-set 50k split")
print("====================")
print(summarize("train", train_df))
print(summarize("val  ", val_df))
print(summarize("test ", test_df))
print("TOTAL:", len(subset), "| NUM_CLASSES:", NUM_CLASSES)
print("\nExamples:\n", train_df.head())


Closed-set 50k split
train: n=39998, ids=1666, min/max per id=23/28
val  : n= 5001, ids=1666, min/max per id=3/4
test : n= 5001, ids=1666, min/max per id=3/4
TOTAL: 50000 | NUM_CLASSES: 1666

Examples:
      image_id  identity split3  label
0  000003.jpg      8692  train   1412
1  000006.jpg      4153  train    574
2  000007.jpg      9040  train   1513
3  000008.jpg      6369  train   1074
4  000009.jpg      3332  train    469


/tmp/ipykernel_557672/1393807774.py:80: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset = subset.groupby("identity", group_keys=False).apply(split_one)


In [3]:
# === Identity classification with ResNet18 (robust setup) ===
from pathlib import Path
from PIL import Image
import os, time, torch, torch.nn as nn
from torch import amp
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
import numpy as np

# ------------------ CONFIG ------------------
ROOT      = Path("/home/anand.ha/NNDL/celeba")
IMG_DIR   = ROOT / "data/raw" / "img_align_celeba"
OUTDIR    = ROOT / "outputs";     OUTDIR.mkdir(parents=True, exist_ok=True)
CKPTDIR   = ROOT / "checkpoints"; CKPTDIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE      = 160
BATCH         = 64            # drop to 48/32 if OOM
EPOCHS        = 10
LR            = 3e-4
WEIGHT_DECAY  = 1e-3          # stronger WD to reduce overfitting
LABEL_SMOOTH  = 0.10
USE_SAMPLER   = True          # class-balanced sampling for train
MIXUP_ALPHA   = 0.2           # 0 to disable MixUp

assert IMG_DIR.is_dir(), f"Images folder missing: {IMG_DIR}"
assert len(train_df) and len(val_df) and len(test_df), "Expected non-empty splits"
# --------------------------------------------

# --------- Transforms (stronger augmentation) ---------
train_tf = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0), ratio=(0.8, 1.25)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandAugment(num_ops=2, magnitude=9),
    T.RandomGrayscale(p=0.10),
    T.ColorJitter(0.15, 0.15, 0.15, 0.05),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    T.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
])
eval_tf = T.Compose([
    T.Resize(int(IMG_SIZE*1.15)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# --------- Dataset ---------
class IdDataset(Dataset):
    def __init__(self, frame, img_dir, transform):
        self.df = frame.reset_index(drop=True)
        self.dir = Path(img_dir)
        self.t   = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r  = self.df.iloc[i]
        x  = Image.open(self.dir / r["image_id"]).convert("RGB")
        x  = self.t(x)
        y  = int(r["label"])
        return x, y

def num_workers_default():
    n = os.cpu_count() or 4
    return max(2, min(8, n-2))

# --------- Optional balanced sampler (train) ---------
if USE_SAMPLER:
    cls_counts = train_df['label'].value_counts().to_dict()
    inv = {c: 1.0/v for c, v in cls_counts.items()}
    sample_weights = train_df['label'].map(inv).astype(float).values
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    train_dl = DataLoader(IdDataset(train_df, IMG_DIR, train_tf),
                          batch_size=BATCH, sampler=sampler, shuffle=False,
                          num_workers=num_workers_default(), pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)
else:
    train_dl = DataLoader(IdDataset(train_df, IMG_DIR, train_tf),
                          batch_size=BATCH, shuffle=True,
                          num_workers=num_workers_default(), pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)

val_dl  = DataLoader(IdDataset(val_df, IMG_DIR, eval_tf),
                     batch_size=BATCH, shuffle=False,
                     num_workers=num_workers_default(), pin_memory=True,
                     persistent_workers=True, prefetch_factor=4)
test_dl = DataLoader(IdDataset(test_df, IMG_DIR, eval_tf),
                     batch_size=BATCH, shuffle=False,
                     num_workers=num_workers_default(), pin_memory=True,
                     persistent_workers=True, prefetch_factor=4)

# --------- Model (ResNet18 + dropout head) ---------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
net.fc = nn.Sequential(
    nn.Dropout(p=0.30),
    nn.Linear(net.fc.in_features, NUM_CLASSES)
)
net.to(device)

# --------- Optim, Loss, AMP, Scheduler ---------
opt     = torch.optim.AdamW(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched   = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)
scaler  = amp.GradScaler("cuda") if device.type=="cuda" else None
crit    = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)

# --------- MixUp helpers ---------
def mixup_data(x, y, alpha=0.2):
    if alpha <= 0: 
        return x, y, y, 1.0
    # sample Beta(alpha, alpha)
    lam = torch.distributions.Beta(alpha, alpha).sample().item()
    lam = max(lam, 1.0 - lam)   # make it symmetric
    idx = torch.randperm(x.size(0), device=x.device)
    return lam * x + (1-lam) * x[idx], y, y[idx], lam

def mixup_criterion(crit, pred, y_a, y_b, lam):
    return lam * crit(pred, y_a) + (1 - lam) * crit(pred, y_b)

# --------- Train/Eval loops ---------
def run_epoch(loader, train: bool, epoch: int):
    net.train(train)
    tot = correct = 0
    loss_sum = 0.0
    t0 = time.time()
    if train: opt.zero_grad(set_to_none=True)
    for step, (xb, yb) in enumerate(loader):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

        if train and MIXUP_ALPHA > 0:
            xb_m, ya, yb2, lam = mixup_data(xb, yb, alpha=MIXUP_ALPHA)
            with amp.autocast("cuda", enabled=(device.type=="cuda")):
                logits = net(xb_m)
                loss   = mixup_criterion(crit, logits, ya, yb2, lam)
        else:
            with amp.autocast("cuda", enabled=(device.type=="cuda")):
                logits = net(xb)
                loss   = crit(logits, yb)

        if train:
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
            else:
                loss.backward()
                opt.step()
            opt.zero_grad(set_to_none=True)

        # metrics (for MixUp we still compute argmax on logits vs yb)
        loss_sum += loss.item() * yb.size(0)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        tot += yb.size(0)

        if step % 200 == 0:
            print(f"[{'train' if train else 'val'} e{epoch} {step}/{len(loader)}] loss={loss.item():.4f}")

    secs_per_step = (time.time() - t0) / max(1, len(loader))
    return loss_sum / max(1, tot), correct / max(1, tot), secs_per_step

# --------- Run training ---------
best_val = 0.0
for ep in range(1, EPOCHS + 1):
    tr_loss, tr_acc, tr_ms = run_epoch(train_dl, True,  ep)
    va_loss, va_acc, va_ms = run_epoch(val_dl,   False, ep)
    sched.step()
    print(f"Epoch {ep:02d} | "
          f"train acc {tr_acc:.3f} loss {tr_loss:.4f} ({tr_ms*1000:.1f} ms/step) | "
          f"val acc {va_acc:.3f} loss {va_loss:.4f} ({va_ms*1000:.1f} ms/step)")
    if va_acc > best_val:
        best_val = va_acc
        torch.save({"epoch": ep,
                    "state_dict": net.state_dict(),
                    "val_acc": va_acc,
                    "num_classes": NUM_CLASSES},
                   CKPTDIR / "id_resnet18_best.pt")
        print("  ↳ saved best checkpoint")

# --------- Test (best checkpoint) ---------
ck = torch.load(CKPTDIR / "id_resnet18_best.pt", map_location=device)
net.load_state_dict(ck["state_dict"])
net.eval()

tot = correct = 0
with torch.no_grad(), amp.autocast("cuda", enabled=(device.type=="cuda")):
    for xb, yb in test_dl:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        pred = net(xb).argmax(1)
        correct += (pred == yb).sum().item()
        tot += yb.size(0)

test_acc = correct / max(1, tot)
print(f"TEST accuracy: {test_acc:.4f} (best val: {best_val:.4f})")


[train e1 0/625] loss=7.8652
[train e1 200/625] loss=7.3322
[train e1 400/625] loss=7.2510
[train e1 600/625] loss=6.6466
[val e1 0/79] loss=6.8549
Epoch 01 | train acc 0.003 loss 7.3068 (68.9 ms/step) | val acc 0.013 loss 6.6644 (35.7 ms/step)
  ↳ saved best checkpoint
[train e2 0/625] loss=6.9300
[train e2 200/625] loss=6.8705
[train e2 400/625] loss=6.1190
[train e2 600/625] loss=5.6395
[val e2 0/79] loss=5.8170
Epoch 02 | train acc 0.030 loss 6.4598 (66.4 ms/step) | val acc 0.063 loss 5.7003 (25.8 ms/step)
  ↳ saved best checkpoint
[train e3 0/625] loss=5.7379
[train e3 200/625] loss=6.3021
[train e3 400/625] loss=5.7125
[train e3 600/625] loss=5.4056
[val e3 0/79] loss=5.1611
Epoch 03 | train acc 0.083 loss 5.8232 (66.5 ms/step) | val acc 0.139 loss 5.1255 (25.1 ms/step)
  ↳ saved best checkpoint
[train e4 0/625] loss=6.7336
[train e4 200/625] loss=5.1390
[train e4 400/625] loss=4.8711
[train e4 600/625] loss=4.7649
[val e4 0/79] loss=4.4126
Epoch 04 | train acc 0.155 loss 5.3183 

/tmp/ipykernel_557672/2001149907.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ck = torch.load(CKPTDIR / "id_resnet18_best.pt", map_location=device)


TEST accuracy: 0.5055 (best val: 0.5041)


In [4]:
# === Identity classification with ResNet34 @ 224 (moderate regularization) ===
from pathlib import Path
from PIL import Image
import os, time, torch, torch.nn as nn
from torch import amp
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
from torchvision.models import resnet34, ResNet34_Weights

ROOT    = Path("/home/anand.ha/NNDL/celeba")
IMG_DIR = ROOT / "data/raw" / "img_align_celeba"
CKPTDIR = ROOT / "checkpoints"; CKPTDIR.mkdir(parents=True, exist_ok=True)

assert IMG_DIR.is_dir()
assert len(train_df) and len(val_df) and len(test_df)

# ---- config
IMG_SIZE = 224
BATCH = 48            # try 64 if memory allows
EPOCHS = 15
LR = 3e-4
WEIGHT_DECAY = 3e-4
LABEL_SMOOTH = 0.05
USE_SAMPLER = True    # class-balanced train sampler

# ---- transforms (moderate)
train_tf = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0), ratio=(0.85, 1.2)),
    T.RandomHorizontalFlip(0.5),
    T.RandAugment(num_ops=2, magnitude=6),
    T.ColorJitter(0.10, 0.10, 0.10, 0.03),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    T.RandomErasing(p=0.15, scale=(0.02, 0.08), ratio=(0.3, 3.3)),
])
eval_tf = T.Compose([
    T.Resize(256),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ---- dataset
class IdDataset(Dataset):
    def __init__(self, frame, img_dir, tfm):
        self.df = frame.reset_index(drop=True)
        self.dir = Path(img_dir); self.t = tfm
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = Image.open(self.dir / r["image_id"]).convert("RGB")
        x = self.t(x)
        return x, int(r["label"])

def nworkers(): 
    n = os.cpu_count() or 4
    return max(2, min(8, n-2))

if USE_SAMPLER:
    counts = train_df["label"].value_counts().to_dict()
    weights = train_df["label"].map({c:1.0/v for c,v in counts.items()}).astype(float).values
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
    train_dl = DataLoader(IdDataset(train_df, IMG_DIR, train_tf),
                          batch_size=BATCH, sampler=sampler, shuffle=False,
                          num_workers=nworkers(), pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)
else:
    train_dl = DataLoader(IdDataset(train_df, IMG_DIR, train_tf),
                          batch_size=BATCH, shuffle=True,
                          num_workers=nworkers(), pin_memory=True,
                          persistent_workers=True, prefetch_factor=4)

val_dl  = DataLoader(IdDataset(val_df, IMG_DIR, eval_tf),
                     batch_size=BATCH, shuffle=False,
                     num_workers=nworkers(), pin_memory=True,
                     persistent_workers=True, prefetch_factor=4)
test_dl = DataLoader(IdDataset(test_df, IMG_DIR, eval_tf),
                     batch_size=BATCH, shuffle=False,
                     num_workers=nworkers(), pin_memory=True,
                     persistent_workers=True, prefetch_factor=4)

# ---- model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
net.fc = nn.Sequential(nn.Dropout(0.15), nn.Linear(net.fc.in_features, NUM_CLASSES))
net.to(device)

# ---- optim/loss/sched
opt   = torch.optim.AdamW(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)
scaler= amp.GradScaler("cuda") if device.type=="cuda" else None
crit  = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)

def run_epoch(loader, train, ep):
    net.train(train)
    tot=correct=0; loss_sum=0.0; t0=time.time()
    if train: opt.zero_grad(set_to_none=True)
    for step,(xb,yb) in enumerate(loader):
        xb,yb = xb.to(device,non_blocking=True), yb.to(device,non_blocking=True)
        with amp.autocast("cuda", enabled=(device.type=="cuda")):
            logits = net(xb); loss = crit(logits,yb)
        if train:
            if scaler: scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            else: loss.backward(); opt.step()
            opt.zero_grad(set_to_none=True)
        loss_sum += loss.item()*yb.size(0)
        pred = logits.argmax(1); correct += (pred==yb).sum().item(); tot += yb.size(0)
        if step%200==0: print(f"[{'train' if train else 'val'} e{ep} {step}/{len(loader)}] loss={loss.item():.4f}")
    return loss_sum/max(1,tot), correct/max(1,tot), (time.time()-t0)/max(1,len(loader))

best_val=0.0
for ep in range(1, EPOCHS+1):
    tr_loss,tr_acc,_ = run_epoch(train_dl, True,  ep)
    va_loss,va_acc,_ = run_epoch(val_dl,   False, ep)
    sched.step()
    print(f"Epoch {ep:02d} | train {tr_acc:.3f}/{tr_loss:.4f} | val {va_acc:.3f}/{va_loss:.4f}")
    if va_acc>best_val:
        best_val=va_acc
        torch.save({"epoch":ep,"state_dict":net.state_dict(),"val_acc":va_acc,"num_classes":NUM_CLASSES},
                   CKPTDIR/"id_resnet34_best.pt")
        print("  ↳ saved best checkpoint")

ck = torch.load(CKPTDIR/"id_resnet34_best.pt", map_location=device)
net.load_state_dict(ck["state_dict"]); net.eval()
tot=correct=0
with torch.no_grad(), amp.autocast("cuda", enabled=(device.type=="cuda")):
    for xb,yb in test_dl:
        xb,yb = xb.to(device,non_blocking=True), yb.to(device,non_blocking=True)
        pred = net(xb).argmax(1); correct += (pred==yb).sum().item(); tot += yb.size(0)
print(f"TEST accuracy: {correct/max(1,tot):.4f} (best val: {best_val:.4f})")


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/anand.ha/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 217MB/s]


[train e1 0/834] loss=7.5181
[train e1 200/834] loss=7.4540
[train e1 400/834] loss=7.4751
[train e1 600/834] loss=7.2060
[train e1 800/834] loss=6.9843
[val e1 0/105] loss=7.2391
Epoch 01 | train 0.001/7.3942 | val 0.002/7.0520
  ↳ saved best checkpoint
[train e2 0/834] loss=7.0556
[train e2 200/834] loss=6.8570
[train e2 400/834] loss=6.7981
[train e2 600/834] loss=6.7698
[train e2 800/834] loss=6.4158
[val e2 0/105] loss=6.7500
Epoch 02 | train 0.004/6.6824 | val 0.006/6.5395
  ↳ saved best checkpoint
[train e3 0/834] loss=6.4990
[train e3 200/834] loss=6.1251
[train e3 400/834] loss=6.3072
[train e3 600/834] loss=6.4014
[train e3 800/834] loss=5.9691
[val e3 0/105] loss=6.2585
Epoch 03 | train 0.012/6.2235 | val 0.016/6.2595
  ↳ saved best checkpoint
[train e4 0/834] loss=5.9431
[train e4 200/834] loss=6.0638
[train e4 400/834] loss=5.7014
[train e4 600/834] loss=5.4180
[train e4 800/834] loss=5.4041
[val e4 0/105] loss=5.4370
Epoch 04 | train 0.038/5.6745 | val 0.056/5.4649
  ↳ sa

/tmp/ipykernel_557672/2564818027.py:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ck = torch.load(CKPTDIR/"id_resnet34_best.pt", map_location=device)


TEST accuracy: 0.6179 (best val: 0.6213)


In [5]:
# === Identity classification on CelebA subset with EfficientNet-B0 ===
from pathlib import Path
from PIL import Image
import os, time, torch, torch.nn as nn, torch.nn.functional as F
from torch import amp
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# --- paths (adjust if needed)
ROOT    = Path("/home/anand.ha/NNDL/celeba")
IMG_DIR = ROOT / "data/raw" / "img_align_celeba"
OUTDIR  = ROOT / "outputs"; OUTDIR.mkdir(parents=True, exist_ok=True)
CKPTDIR = ROOT / "checkpoints"; CKPTDIR.mkdir(parents=True, exist_ok=True)

# assumes you already have train_df, val_df, test_df, NUM_CLASSES built
assert len(train_df) and len(val_df) and len(test_df), "Expected non-empty splits"

# --- transforms
IMG_SIZE = 160
train_tf = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0), ratio=(0.8, 1.25)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandAugment(num_ops=2, magnitude=9),
    T.RandomGrayscale(p=0.10),
    T.ColorJitter(0.15, 0.15, 0.15, 0.05),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    T.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3))
])

eval_tf = T.Compose([
    T.Resize(int(IMG_SIZE*1.15)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# --- dataset class
class IdDataset(Dataset):
    def __init__(self, frame, img_dir, transform):
        self.df = frame.reset_index(drop=True)
        self.dir = Path(img_dir); self.t = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = Image.open(self.dir / r["image_id"]).convert("RGB")
        x = self.t(x)
        y = int(r["label"])
        return x, y

def num_workers_default():
    n = os.cpu_count() or 4
    return max(2, min(8, n-2))

BATCH = 64
train_dl = DataLoader(IdDataset(train_df, IMG_DIR, train_tf),
                      batch_size=BATCH, shuffle=True,
                      num_workers=num_workers_default(), pin_memory=True,
                      persistent_workers=True, prefetch_factor=4)
val_dl   = DataLoader(IdDataset(val_df,   IMG_DIR, eval_tf),
                      batch_size=BATCH, shuffle=False,
                      num_workers=num_workers_default(), pin_memory=True,
                      persistent_workers=True, prefetch_factor=4)
test_dl  = DataLoader(IdDataset(test_df,  IMG_DIR, eval_tf),
                      batch_size=BATCH, shuffle=False,
                      num_workers=num_workers_default(), pin_memory=True,
                      persistent_workers=True, prefetch_factor=4)

# --- model (EfficientNet-B0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
net.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(net.classifier[1].in_features, NUM_CLASSES)
)
net.to(device)

# --- optimizer/loss/amp
opt = torch.optim.AdamW(net.parameters(), lr=3e-4, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=15)
scaler = amp.GradScaler("cuda") if device.type=="cuda" else None
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# --- training loop
def run_epoch(loader, train: bool, epoch: int):
    net.train(train)
    tot, correct, loss_sum = 0, 0, 0.0
    if train: opt.zero_grad(set_to_none=True)
    t0 = time.time()
    for step, (xb, yb) in enumerate(loader):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        with amp.autocast("cuda", enabled=(device.type=="cuda")):
            logits = net(xb)
            loss   = criterion(logits, yb)
        if train:
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(opt); scaler.update()
            else:
                loss.backward(); opt.step()
            opt.zero_grad(set_to_none=True)
        loss_sum += loss.item() * yb.size(0)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        tot += yb.size(0)
        if step % 200 == 0:
            print(f"[{'train' if train else 'val'} e{epoch} {step}/{len(loader)}] loss={loss.item():.4f}")
    sec = time.time()-t0
    return loss_sum/max(1,tot), correct/max(1,tot), sec/len(loader)

# --- train for 15 epochs
best_val = 0.0
EPOCHS = 15
for ep in range(1, EPOCHS+1):
    tr_loss, tr_acc, tr_ms = run_epoch(train_dl, True,  ep)
    va_loss, va_acc, va_ms = run_epoch(val_dl,   False, ep)
    sched.step()
    print(f"Epoch {ep:02d} | train {tr_acc:.3f}/{tr_loss:.4f} | val {va_acc:.3f}/{va_loss:.4f}")
    if va_acc > best_val:
        best_val = va_acc
        torch.save({"epoch":ep, "state_dict":net.state_dict(), "val_acc":va_acc,
                    "num_classes": NUM_CLASSES},
                   CKPTDIR/"id_efficientnet_b0_best.pt")
        print("  ↳ saved best checkpoint")

# --- test eval
ck = torch.load(CKPTDIR/"id_efficientnet_b0_best.pt", map_location=device)
net.load_state_dict(ck["state_dict"]); net.eval()
tot, correct = 0, 0
with torch.no_grad(), amp.autocast("cuda", enabled=(device.type=="cuda")):
    for xb, yb in test_dl:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        pred = net(xb).argmax(1)
        correct += (pred == yb).sum().item()
        tot += yb.size(0)
test_acc = correct / max(1, tot)
print(f"TEST accuracy: {test_acc:.4f} (best val: {best_val:.4f})")


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /home/anand.ha/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 90.2MB/s]


[train e1 0/625] loss=7.4737
[train e1 200/625] loss=7.3040
[train e1 400/625] loss=6.8235
[train e1 600/625] loss=6.1865
[val e1 0/79] loss=5.7852
Epoch 01 | train 0.014/6.9203 | val 0.080/5.6904
  ↳ saved best checkpoint
[train e2 0/625] loss=5.6545
[train e2 200/625] loss=5.6668
[train e2 400/625] loss=5.2956
[train e2 600/625] loss=4.9763
[val e2 0/79] loss=4.3553
Epoch 02 | train 0.114/5.3700 | val 0.264/4.4909
  ↳ saved best checkpoint
[train e3 0/625] loss=4.3242
[train e3 200/625] loss=4.5020
[train e3 400/625] loss=4.1200
[train e3 600/625] loss=4.3903
[val e3 0/79] loss=3.5428
Epoch 03 | train 0.265/4.4107 | val 0.415/3.7622
  ↳ saved best checkpoint
[train e4 0/625] loss=3.8484
[train e4 200/625] loss=3.7023
[train e4 400/625] loss=3.3510
[train e4 600/625] loss=3.6288
[val e4 0/79] loss=3.1287
Epoch 04 | train 0.397/3.7501 | val 0.520/3.3204
  ↳ saved best checkpoint
[train e5 0/625] loss=3.4909
[train e5 200/625] loss=3.1059
[train e5 400/625] loss=3.3256
[train e5 600/625

/tmp/ipykernel_557672/2211948592.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ck = torch.load(CKPTDIR/"id_efficientnet_b0_best.pt", map_location=device)


TEST accuracy: 0.7295 (best val: 0.7303)


In [12]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
from pathlib import Path

# === Paths (edit IMG_PATH as you like) ===
CKPT_PATH = Path("/home/anand.ha/NNDL/celeba/checkpoints/id_efficientnet_b0_best.pt")
IMG_PATH  = Path("/home/anand.ha/NNDL/celeba/data/raw/img_align_celeba/000098.jpg")  # change to any image

# === Load checkpoint first to get NUM_CLASSES if available ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt = torch.load(CKPT_PATH, map_location=device)
num_classes = ckpt.get("num_classes", 666)  # fallback if not saved

# === Build model EXACTLY like training ===
net = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
in_features = net.classifier[1].in_features
# training used: classifier[1] = Sequential(Dropout, Linear)
net.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(in_features, num_classes)
)
net.to(device)

# Load weights (keep strict=True since we now match the arch)
net.load_state_dict(ckpt["state_dict"])
net.eval()

# === Preprocessing (same as eval_tf) ===
IMG_SIZE = 160
transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# === Load image and predict ===
img = Image.open(IMG_PATH).convert("RGB")
x = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = net(x)
    pred_class = logits.argmax(dim=1).item()

print(f"Predicted identity label index: {pred_class}")


/tmp/ipykernel_557672/90107654.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(CKPT_PATH, map_location=device)


Predicted identity label index: 1568
